In [1]:
import asyncio
from serial_interface.serial_interface import TwidSerialInterfaceProtocol, TelemetryFrame, run_test, ControlType, start_protocol

import os

###SERIAL CONFIGURATION for esp32
SERIAL_PORT = 'COM9'
SERIAL_BAUD_RATE = 1000000

DATA_DIR_PATH = os.path.join(os.getcwd(),'data')

twid = await start_protocol(asyncio.get_event_loop(), SERIAL_PORT, SERIAL_BAUD_RATE)

port opened SerialTransport(<_WindowsSelectorEventLoop running=True closed=False debug=False>, <serial_interface.serial_interface.TwidSerialInterfaceProtocol object at 0x0000029B9C51E870>, Serial<id=0x29b9cda1990, open=True>(port='COM9', baudrate=1000000, bytesize=8, parity='N', stopbits=1, timeout=0, xonxoff=False, rtscts=False, dsrdtr=False))


In [2]:
#control parameter fitting
import scipy.optimize
import scipy.signal
import numpy as np

import matplotlib.pyplot as plt
import pandas

import control.matlab as control

#MOTOR EMPIRCAL PARAMS

#voltage supply
PWM_AMPLITUDE = 12
DC_STEPS = [128, 256, 512, 1024]

#second order model
def so_model(x,K,wn,zeta):
    tf = control.tf([K],[1,2*wn*zeta,wn**2])
    yout,T = control.step(tf, T=x)
    return yout[-1]

In [19]:
#reset
await twid.esp32_reboot()
twid.clear_frames()
await twid.update_telem_sample_rate(5)
await twid.update_control_type(ControlType.NO_CTRL)
await twid.wait_for_param('control_type', ControlType.NO_CTRL)
await twid.update_dutycycle(0)
await twid.wait_for_param('pwm_duty_cycle',0)

step_input = 512
#apply step to 512/1024
await twid.update_dutycycle(step_input)
#collect all data for next 5 seconds
dataseries = await twid.collect_telem(5)
print(dataseries)
#stop motor
await twid.motor_stop()

#save as a nice csv using pandas
dataseries.to_csv(os.path.join(DATA_DIR_PATH,f'current_step_response_{step_input}.csv'), index=False)

#collect info from start of step only
ts = dataseries['timestamp_ms'][dataseries['pwm_duty_cycle'] >= step_input]
current = dataseries['filtered_current'][dataseries['pwm_duty_cycle'] >= step_input]
vel = dataseries['filtered_velocity'][dataseries['pwm_duty_cycle'] >= step_input]

#apply curve fitting to get second order params using scipy
[K, wn, zeta],_ = scipy.optimize.curve_fit(so_model, ts.to_numpy(), current.to_numpy())
output_str = f'Step response estimated second order parameters:\n\
    K:{K}\twn:{wn}\tzeta:{zeta}\n'
print(output_str)

[K, wn, zeta],_ = scipy.optimize.curve_fit(so_model, ts.to_numpy(), vel.to_numpy())
output_str = f'Step response estimated second order parameters:\n\
    K:{K}\twn:{wn}\tzeta:{zeta}\n'
print(output_str)


    timestamp_ms  loop_dt  control_dt  read_dt  pwm_duty_cycle  pwm_frequency  \
0          451.0   1000.0        28.0     66.0           512.0           -0.0   
1          456.0    992.0        28.0     66.0           512.0           -0.0   
2          461.0   1000.0        28.0     66.0           512.0           -0.0   
3          466.0   1000.0        28.0     66.0           512.0           -0.0   
4          471.0   1000.0        28.0     66.0           512.0           -0.0   
..           ...      ...         ...      ...             ...            ...   
90         991.0   1000.0        16.0     57.0           512.0           -0.0   
91         996.0   1012.0        17.0     57.0           512.0           -0.0   
92        1001.0    994.0        25.0     65.0           512.0           -0.0   
93        1006.0    993.0        25.0     65.0           512.0           -0.0   
94        1011.0    999.0        16.0     57.0           512.0           -0.0   

    position  velocity  fi

ValueError: Time steps are not equally spaced.